In [5]:
import re
import os
import subprocess
source = "Jio"
# destination = ["google.txt","alibaba.txt","apple.txt","bitcoin.txt","goldmansachs.txt","ntt.txt","spotify.txt","vodacom.txt","wikipedia.txt","oracle.txt"]
destination = ["nvidia.txt"]
def getdata(source,destination):
    file = open(source+"/"+destination,"r")
    data = {
        "AS NUMBER" : [],
        "IP ADDRESS": [],
        "IP RANGE" : [],
        "LOCATION" : [],
        "ORGANISATION": []
    }
    for line in file.readlines():

    
        s = re.findall('\[AS[0-9]+',line)
        
        if (len(s)>0 and s[0][1] != "*"):
            
            ips = re.findall('[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+',line)

            statement = "./IPGeoLocation/ipgeolocation.py -t {} ".format(ips[0])
            proc = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc.communicate()
            out = out.decode()
            city = re.findall('City:[^\n]*\\n',out)
            city = re.findall('[A-Z][a-zA-Z ]*',city[0])
            country= re.findall('Country:[^\n]*\\n',out)
            country = re.findall('[A-Z][a-zA-Z ]*',country[0])
            org= re.findall('Organization:[^\n]*\\n',out)
            org = re.findall('[A-Z][a-zA-Z ]*',org[0])
            if (len(org) >=2):
                org = org[1]
            else:
                org = ""
            if (len(city) >=2):
                city = city[1]
            else:
                city = ""
            if (len(country) >=2):
                country = country[1]
            else:
                country = ""
            
            loc = city+'/'+country
            statement = "whois -a {}".format(ips[0])
            proc2 = subprocess.Popen([statement],stdout=subprocess.PIPE,shell=True)
            (out,err) = proc2.communicate()
            out = out.decode()
            iprange = re.findall('inetnum:[ \t]*[-0-9\. ]*|NetRange:[ \t]*[-0-9\. ]*',out)
            if (len(iprange) > 0):
                iprange = iprange[0].split(":")[1].lstrip()
            else:
                iprange = ""
                
            
            data["IP RANGE"].append(iprange)
            data["AS NUMBER"].append(s[0][1:])
            data["IP ADDRESS"].append(ips[0])
            data["ORGANISATION"].append(org)
            data["LOCATION"].append(loc)
            
    file.close()
    return data


In [6]:
# getdata(source,destination[0])
destination_dict = {}
for d in destination:
    print(d)
    destination_dict[d] = getdata(source,d)
destination_dict

nvidia.txt


{'nvidia.txt': {'AS NUMBER': ['AS64049',
   'AS64049',
   'AS64049',
   'AS64049',
   'AS14618',
   'AS14618'],
  'IP ADDRESS': ['103.198.140.176',
   '103.198.140.213',
   '103.198.140.43',
   '103.198.140.43',
   '34.194.97.138',
   '34.194.97.138'],
  'IP RANGE': ['103.198.140.0 - 103.198.140.255',
   '103.198.140.0 - 103.198.140.255',
   '103.198.140.0 - 103.198.140.255',
   '103.198.140.0 - 103.198.140.255',
   '34.192.0.0 - 34.255.255.255',
   '34.192.0.0 - 34.255.255.255'],
  'LOCATION': ['Singapore/Singapore',
   'Singapore/Singapore',
   'Singapore/Singapore',
   'Singapore/Singapore',
   'Ashburn/United States',
   'Ashburn/United States'],
  'ORGANISATION': ['', '', '', '', 'AWS EC', 'AWS EC']}}

In [3]:
destination_dict

{}

In [129]:
import pandas as pd

for d in destination:
    df = pd.DataFrame(destination_dict[d])
    df.to_excel(source +"/"+"structured" + "/" + d[:-3]+"xlsx")